In [ ]:

!pip install llama-index-vector-stores-milvus llama-index-llms-ollama llama-index-embeddings-huggingface
!pip install pymilvus

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing insta

In [ ]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=5a7eac13abdda8e5d8ea4f8be207d208f94a3fe6164d93a7ae382d1541ada145
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [ ]:
!pip install llama-index
!pip install llama-index-readers-file


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.4/187.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 24.4 MB/s eta 0:00:00


In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Define the path to the already uploaded PDF
pdf_file_path = "/content/drive/MyDrive/LLMs System Design and Applications (2024-Draft-1).pdf"

# Instantiate Milvus vector database
vector_store = MilvusVectorStore(
    uri="./milvus_rag_demo.db", dim=768, overwrite=True
)

# Load the embedding model from HuggingFace
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate Llama model
llm = Ollama(model="llama3", temperature=0.1, request_timeout=480.0)


DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 93975a4ef84e4229a7868a27717d44d8
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: llamacollection
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: llamacollection


In [ ]:

# Ingest the uploaded PDF using SimpleDirectoryReader
pdf_document = SimpleDirectoryReader(
    input_files=[pdf_file_path]
).load_data()

print("Number of Input documents:", len(pdf_document))

# Bind LLM and embedding model in the RAG pipeline settings
Settings.llm = llm
Settings.embed_model = embedding_model

# Customize chunk size and overlap
Settings.chunk_size = 128
Settings.chunk_overlap = 64

# Create index and load PDF chunks into Milvus vector database
index = VectorStoreIndex.from_documents(pdf_document)
print("Number of nodes:", len(index.docstore.docs))

# Prepare query engine
query_engine = index.as_query_engine()

# Example query over the uploaded research paper
query = ""
result = query_engine.query(query)

# Display result
print(result)

In [ ]:

from llama_index.core import SimpleDirectoryReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from pymilvus import MilvusClient

In [ ]:
%pip install llama-index-vector-stores-chroma
%pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Define the path to the already uploaded PDF
pdf_file_path = "/content/LLMs System Design and Applications (2024-Draft-1).pdf"

# Replace with your Milvus server's host and port
milvus_host = "your-milvus-host"
milvus_port = "your-milvus-port"

# Instantiate Milvus vector database
vector_store = MilvusVectorStore(
    uri=f"tcp://{milvus_host}:{milvus_port}", dim=768, overwrite=True
)

# Load the embedding model from HuggingFace
embedding_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

# Instantiate Llama model
llm = Ollama(model="llama3", temperature=0.1, request_timeout=480.0)

# Ingest the uploaded PDF using SimpleDirectoryReader
pdf_document = SimpleDirectoryReader(
    input_files=[pdf_file_path]
).load_data()

print("Number of Input documents:", len(pdf_document))

# Bind LLM and embedding model in the RAG pipeline settings
Settings.llm = llm
Settings.embed_model = embedding_model

# Customize chunk size and overlap
Settings.chunk_size = 128
Settings.chunk_overlap = 64

# Create index and load PDF chunks into Milvus vector database
index = VectorStoreIndex.from_documents(pdf_document)
print("Number of nodes:", len(index.docstore.docs))

# Prepare query engine
query_engine = index.as_query_engine()

# Example query over the uploaded research paper
query = "What is the benefit of multi-head attention instead of single-head attention?"
result = query_engine.query(query)

# Display result
print(result)
